In [1]:
import lxml.html
import requests

# Parse vybory.izbirkom.ru page
izbirkom_page = requests.get("http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&root=1&tvd=100100084849066&vrn=100100084849062&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=100100084849066&type=227")
parsed_body = lxml.html.fromstring(izbirkom_page.text)

# XPath of an element with links
xpath = "/table[2]/tr[4]/table[6]/td[2]/a"
xpath = xpath.replace('/','//')

table = parsed_body.xpath(xpath)

In [2]:
# An array of regions and links
region_links = []
for element in table:
    region_links.append((element.text_content(), element.get('href')))
region_links[0]

('Республика Адыгея (Адыгея)',
 'http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&tvd=100100084849066&vrn=100100084849062&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=100100084849067&type=227')

In [5]:
from tqdm import tqdm_notebook

# Parse territorial election comission
tik_links = []
for (region, link) in tqdm_notebook(region_links):
    region_page = requests.get(link)
    parsed_body = lxml.html.fromstring(region_page.text)
    
    xpath = "/nobr/a"
    xpath = xpath.replace('/','//')
    
    table = parsed_body.xpath(xpath)
    for element in table:
        title = element.text_content()
        tik_link = element.get('href')
        tik_links.append((region, title, tik_link))
tik_links[0]

('Республика Адыгея (Адыгея)',
 'Адыгейская',
 'http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&tvd=100100084849067&vrn=100100084849062&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=2012000364371&type=227')

In [7]:
# Parse local election comission
uik_links = []
for (region, tik, link) in tqdm_notebook(tik_links):
    tik_page = requests.get(link)
    parsed_body = lxml.html.fromstring(tik_page.text)
    
    xpath = "/table[2]/tr[2]/td/a"
    xpath = xpath.replace('/','//')
    
    table = parsed_body.xpath(xpath)
    if len(table) == 1:
        uik_link = table[0].get('href')
        uik_links.append((region, tik, uik_link))
uik_links[0]

('Республика Адыгея (Адыгея)',
 'Адыгейская',
 'http://www.vybory.izbirkom.ru/region/izbirkom?action=show&global=true&root=12000009&tvd=2012000364371&vrn=100100084849062&prver=0&pronetvd=null&region=1&sub_region=1&type=227&vibid=2012000364371')

In [10]:
import numpy as np

# Parse uik page
i = 0
for (region, tik, uik_link) in tqdm_notebook(uik_links):
    
    uik_page =  requests.get(uik_link)
    parsed_body = lxml.html.fromstring(uik_page.text)
    
    uik_names = parsed_body.xpath('//table[6]//td[2]//tr[1]')
    UIK_table = parsed_body.xpath('//table[6]//td[2]//tr')
    
    uik_names_list = []
    for uik_name in uik_names[0].getchildren():
        uik_names_list.append(uik_name.text_content())

    table = []
    num_rows = len(uik_names_list)
    table += [[region] * num_rows]
    table += [[tik]  * num_rows]
    table += [uik_names_list]

    for tr in UIK_table:
        this_row = []
        if tr.get('valign'):
            continue
        for td in tr.getchildren():
            if len(td.getchildren()[0].getchildren()) == 0:
                this_row += ['empty_row']
                break
            this_row += [int(td.getchildren()[0].getchildren()[0].text_content())]
        if this_row != ['empty_row']:
            table.append(this_row)
    dataframe = pd.DataFrame(np.array(table).T)
    i += 1
    dataframe.to_csv(f'uiks/uik_{i}.csv', header = False, index = False)

In [11]:
csv_file = pd.read_csv("uiks/uik_2775.csv")
csv_file

,Ямало-Ненецкий автономный округ,Шурышкарская,УИК №1201,218,220,25,178,7,10,32,...,0,0.1,2,4,6,187,3.1,2.1,3.2,0.2
0,Ямало-Ненецкий автономный округ,Шурышкарская,УИК №1202,960,1007,12,880,45,70,57,...,0,0,2,23,31,861,2,3,3,0
1,Ямало-Ненецкий автономный округ,Шурышкарская,УИК №1203,333,355,50,268,8,29,58,...,0,0,1,5,7,306,2,1,0,1
2,Ямало-Ненецкий автономный округ,Шурышкарская,УИК №1204,2459,2430,30,2226,121,53,151,...,0,0,8,41,72,2210,10,9,7,4
3,Ямало-Ненецкий автономный округ,Шурышкарская,УИК №1205,349,353,69,259,9,16,78,...,0,0,2,4,8,312,2,1,2,2
4,Ямало-Ненецкий автономный округ,Шурышкарская,УИК №1206,791,817,202,553,14,48,216,...,0,0,2,7,17,726,2,3,2,3
5,Ямало-Ненецкий автономный округ,Шурышкарская,УИК №1207,308,335,38,256,9,32,47,...,0,0,0,3,5,286,3,1,2,1
6,Ямало-Ненецкий автономный округ,Шурышкарская,УИК №1208,518,549,24,449,19,57,43,...,0,0,1,14,21,445,0,0,2,4


In [15]:
from os import listdir

with open('result.csv', mode='w+', encoding='utf8') as f:
    for file in tqdm_notebook(listdir('uiks/')):
        with open('uiks/' + file, mode='r', encoding='utf8') as report:
            for line in report:
                f.write(line)

In [16]:
df = pd.read_csv('result.csv')
df

,Республика Адыгея (Адыгея),Адыгейская,УИК №1,2256,2181,0,2107,62,12,62.1,...,0.1,0.2,0.3,137,32,1977,14,0.4,1,5
0,Республика Адыгея (Адыгея),Адыгейская,УИК №2,2700,2633,0,2575,41,17,41,...,0,0,15,86,65,2389,13,5,6,15
1,Республика Адыгея (Адыгея),Адыгейская,УИК №3,2858,2752,0,2664,75,13,75,...,0,0,1,62,13,2645,6,3,4,0
2,Республика Адыгея (Адыгея),Адыгейская,УИК №4,2066,2034,0,1857,142,35,142,...,0,0,5,288,12,1642,21,6,2,2
3,Республика Адыгея (Адыгея),Адыгейская,УИК №5,700,714,0,676,11,27,11,...,0,0,2,44,6,624,6,0,0,2
4,Республика Адыгея (Адыгея),Адыгейская,УИК №6,1044,1036,0,954,68,14,68,...,0,0,1,38,21,952,2,2,2,1
5,Республика Алтай,Горно-Алтайская городская,УИК №1,2120,2000,0,1299,35,666,35,...,0,0,8,353,72,848,13,8,4,10
6,Республика Алтай,Горно-Алтайская городская,УИК №2,2181,2000,0,1421,33,546,33,...,0,0,3,341,74,995,14,7,2,5
7,Республика Алтай,Горно-Алтайская городская,УИК №3,1748,1700,0,1081,30,589,30,...,0,0,4,257,69,733,14,3,11,11
8,Республика Алтай,Горно-Алтайская городская,УИК №4,2952,2800,0,1618,124,1058,124,...,0,0,7,469,102,1098,18,7,9,17
9,Республика Алтай,Горно-Алтайская городская,УИК №5,2470,2200,0,1564,15,621,15,...,0,0,11,356,78,1083,18,9,3,11


In [21]:
import pyspark
import os

from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

os.environ["PYSPARK_PYTHON"]="python3"
spark = SparkSession.builder.appName("PySpark").getOrCreate()

file = 'result.csv'

schema= StructType([
            StructField("region",              StringType(), False), 
            StructField("tik",                 StringType(), False), 
            StructField("uik",                 StringType(), False), 
            StructField("total_voters_number", IntegerType(), False),
            StructField("number_of_blanks",    IntegerType(), False),
            StructField("early_blanks",        IntegerType(), False),
            StructField("indoors_blanks",      IntegerType(), False),
            StructField("outdoors_blanks",     IntegerType(), False),
            StructField("canceled_blanks",     IntegerType(), False),
            StructField("out_box_blanks",      IntegerType(), False),
            StructField("in_box_blanks",       IntegerType(), False),
            StructField("invalid_blanks",      IntegerType(), False),
            StructField("valid_blanks",        IntegerType(), False),
            StructField("lost_blanks",         IntegerType(), False),
            StructField("uncounted_blanks",    IntegerType(), False),
            StructField("baburin",             IntegerType(), False),
            StructField("grudinin",            IntegerType(), False),
            StructField("zhirinovsky",         IntegerType(), False),
            StructField("putin",               IntegerType(), False),
            StructField("sobchak",             IntegerType(), False),
            StructField("suraykin",            IntegerType(), False),
            StructField("titov",               IntegerType(), False),
            StructField("yavlinsky",           IntegerType(), False)]
        )

df = spark.read.csv(file, schema=schema)
df.createOrReplaceTempView('election')

In [22]:
# Явка по всем регионам по убыванию
turnout = spark.sql(
    '''SELECT region, (SUM(out_box_blanks) + SUM(in_box_blanks)) / SUM(total_voters_number) as turnout 
    FROM election
    GROUP BY region
    ORDER BY turnout DESC''')

turnout.show()

+--------------------+------------------+
|              region|           turnout|
+--------------------+------------------+
|     Республика Тыва|0.9362542974951742|
|Ямало-Ненецкий ав...|0.9187213306617982|
|Кабардино-Балкарс...| 0.916978754085207|
|Чеченская Республика|0.9150929703298174|
|Республика Северн...|0.8993704831730909|
| Республика Дагестан|0.8744351419820495|
|Карачаево-Черкесс...|0.8734380939009899|
| Кемеровская область|0.8307556958224817|
|Чукотский автоном...|0.8226051697921947|
|Республика Ингушетия|0.8195937165135314|
|    Брянская область| 0.796597680757152|
|   Тюменская область|0.7888296737637491|
| Республика Мордовия|0.7783648793754097|
|  Краснодарский край|0.7783466166202626|
|Республика Татарс...|0.7736526548202729|
|Чувашская Республ...|0.7621061419670118|
|Республика Башкор...|0.7542120065745193|
|  Республика Бурятия|0.7519133382510174|
|Республика Адыгея...| 0.743059515324305|
| Ставропольский край|0.7377007114684981|
+--------------------+------------

In [38]:
# Избирательные участки, на которых кандидат получил наибольший результат 
my_candidate = 'putin'
best_uiks = spark.sql(
    f'''SELECT region, tik, uik, {my_candidate}/total_voters_number as percent 
    FROM election
    WHERE total_voters_number > 300
    ORDER BY percent DESC''')
best_uiks.show(10)

+--------------------+--------------------+--------+------------------+
|              region|                 tik|     uik|           percent|
+--------------------+--------------------+--------+------------------+
| Кемеровская область|Прокопьевск, Цент...|УИК №876|               1.0|
|     Республика Тыва|      Бай-Тайгинская| УИК №43|               1.0|
|     Республика Тыва|           Эрзинская|УИК №180|               1.0|
|     Республика Тыва|        Улуг-Хемская|УИК №159|               1.0|
|Республика Татарс...|         Апастовская|УИК №909|0.9976415094339622|
|     Республика Тыва|         Тес-Хемская|УИК №145|0.9973045822102425|
|     Республика Тыва|      Бай-Тайгинская| УИК №45|0.9972714870395635|
|     Республика Тыва|        Улуг-Хемская|УИК №165| 0.996594778660613|
|     Республика Тыва|        Улуг-Хемская|УИК №163|0.9963325183374083|
|Республика Башкор...|    Уфа, Калининская| УИК №98|0.9956331877729258|
+--------------------+--------------------+--------+------------

In [41]:
# Регионы, где разница между ТИК с наибольшей явкой и наименьшей максимальна
tik_turnout = spark.sql(
    f'''SELECT region, MAX(turnout) - MIN(turnout) as diff
    FROM
        (SELECT region, tik, (SUM(out_box_blanks) + SUM(in_box_blanks)) / SUM(total_voters_number) as turnout
        FROM election
        GROUP BY region, tik)
    GROUP BY region
    ORDER BY diff DESC''')
tik_turnout.show(3)

+--------------------+-------------------+
|              region|               diff|
+--------------------+-------------------+
|Архангельская обл...|0.49859164840684894|
| Сахалинская область| 0.4842508035844767|
| Республика Дагестан| 0.4644410241767303|
+--------------------+-------------------+
only showing top 3 rows



In [45]:
# Дисперсия по явке для каждого региона
variance = spark.sql(
    f'''SELECT region, STDDEV((out_box_blanks + in_box_blanks) / total_voters_number) as variance
    FROM election 
    GROUP BY region
    ORDER BY variance DESC''')
variance.show()

+--------------------+-------------------+
|              region|           variance|
+--------------------+-------------------+
| Сахалинская область|0.20188798623588156|
|     Камчатский край|0.16873475777512123|
|  Мурманская область|0.16742131453895345|
|     Приморский край|0.16639015831512335|
|Республика Адыгея...|0.16333249598843577|
|   Самарская область| 0.1562883627711599|
| Саратовская область|0.15426531267153373|
| Республика Калмыкия|0.15350844523090657|
| Воронежская область|0.14975754071870498|
| Магаданская область|0.14387651017723133|
|Республика Татарс...|0.14172353736111934|
|    Липецкая область|0.14096365252591136|
|Архангельская обл...| 0.1409455295612835|
|  Краснодарский край| 0.1394657822552809|
|    Хабаровский край|0.13811595897262202|
|Белгородская область| 0.1375828241478792|
| Ставропольский край|0.13411190491014102|
|  Республика Бурятия| 0.1330825305273712|
|  Ростовская область|0.13138651268427387|
|Чувашская Республ...|0.13041627601227343|
+----------

In [56]:
# Количество УИК, на которых кандидат получил данный результат
candidates = ['baburin', 'grudinin', 'zhirinovsky', 'putin', 'sobchak', 'suraykin', 'titov', 'yavlinsky']
     
for candidate in candidates:
    hist_data = spark.sql(
        f'''SELECT result, COUNT(result) as uiks_number FROM
                (SELECT uik, int(100 * {candidate} / total_voters_number) as result
                FROM election)
            GROUP BY result
            ORDER BY uiks_number DESC''')
    print(candidate)
    hist_data.show()

baburin
+------+-----------+
|result|uiks_number|
+------+-----------+
|     0|      92025|
|     1|       4431|
|     2|        513|
|     3|        152|
|     4|         70|
|     5|         30|
|     6|         21|
|     7|         16|
|    10|          8|
|     8|          8|
|     9|          5|
|    14|          3|
|    11|          2|
|    16|          2|
|    20|          2|
|    44|          1|
|    23|          1|
|    13|          1|
|    12|          1|
|    17|          1|
+------+-----------+
only showing top 20 rows

grudinin
+------+-----------+
|result|uiks_number|
+------+-----------+
|     7|      12752|
|     6|      12084|
|     8|      10966|
|     5|       9500|
|     9|       7860|
|     4|       6942|
|    10|       5740|
|     3|       5235|
|     2|       4332|
|    11|       3943|
|     1|       3700|
|    12|       3022|
|     0|       2449|
|    13|       2267|
|    14|       1534|
|    15|       1185|
|    16|        906|
|    17|        632|
|    18|    